This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

Bb3LTwcLLyXsg4QDqxps


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
request_url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json'
payload = {'rows':'10', 'api_key':API_KEY}
response = requests.get(request_url, params=payload)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(response.url)
response.content

https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?rows=10&api_key=Bb3LTwcLLyXsg4QDqxps


b'{"dataset_data":{"limit":10,"transform":null,"column_index":null,"column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"start_date":"2000-06-07","end_date":"2020-12-01","frequency":"daily","data":[["2020-12-01",112.2,112.2,111.5,112.0,null,51.0,5703.0,null,null,null],["2020-11-30",111.0,113.6,111.0,112.1,null,315.0,35111.5,null,null,null],["2020-11-27",111.0,111.5,109.5,111.2,null,155.0,17168.5,null,null,null],["2020-11-26",111.3,112.0,110.5,111.6,null,254.0,28367.5,null,null,null],["2020-11-25",111.4,111.4,110.1,110.1,null,294.0,32435.8,null,null,null],["2020-11-24",116.1,116.8,110.5,113.5,null,3184.0,361404.4,null,null,null],["2020-11-23",118.0,118.8,116.3,116.3,null,1590.0,187525.0,null,null,null],["2020-11-20",116.5,118.0,116.5,117.5,null,295.0,34736.0,null,null,null],["2020-11-19",115.6,115.6,115.6,115.6,null,0.0,0.0,null,null,null],["2020-11-18",115.8,115.8,115.0,115.0,null,25.0,2875

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
payload = {'start_date':'2017-01-01', 'end_date':'2017-12-31'}
response = requests.get(request_url, params=payload)
data_dict = response.json()
print(data_dict["dataset_data"]["column_names"])
# print(data_dict["dataset_data"]["data"][0])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [6]:
open_idx = 1
high_idx = 2 
low_idx = 3
close_idx = 4
volume_idx = 6

# Calulcate the lists necessary to answet the questions above
op = []
close = []
day_diff = []
two_day_change = []
volume = []

for row in data_dict["dataset_data"]["data"]:
    if row[open_idx] is not None:
        op.append(row[open_idx])
    if row[close_idx] is not None:
        if len(close) > 0:
            two_day_change.append(row[close_idx] - close[-1])
        close.append(row[close_idx])
        if row[open_idx] is not None:
            day_diff.append(abs(row[close_idx] - row[open_idx]))
    if row[volume_idx] is not None:
        volume.append(row[volume_idx])

In [7]:
# Calculate what the highest and lowest opening prices were for the stock in this period.
print("Highest opening prices: {:.2f}".format(max(op)))
print("Lowest opening prices: {:.2f}".format(min(op)))

# What was the largest change in any one day (based on High and Low price)?
print("Largest change in any one day: {:.2f}".format(max(day_diff)))

# What was the largest change between any two days (based on Closing Price)?
print("Largest change btw. any two days: {:.2f}".format(max(two_day_change)))

volume.sort()
# What was the average daily trading volume during this year?
print("Average daily trading volume: {:.2f}".format(sum(volume)/len(volume)))

# What was the median trading volume during this year.
median_idx = int(len(volume)/2)
median = volume[median_idx]

if len(volume) % 2 == 0:
    median = (volume[median_idx] + volume[median_idx - 1])/2

print("Median daily trading volume: {:.2f}".format(median))


Highest opening prices: 53.11
Lowest opening prices: 34.00
Largest change in any one day: 1.69
Largest change btw. any two days: 2.56
Average daily trading volume: 89124.34
Median daily trading volume: 76286.00
